<a href="https://colab.research.google.com/github/Habibu-Ahmad/Deepfake_Detection/blob/main/Deeplearning_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

**Imports**

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

**Data Loading and Processing**

In [ ]:
# Define paths to your image folders
base_path = "/content/drive/MyDrive/Deepfake_Images"
train_real_path = os.path.join(base_path, "real_train")
train_fake_path = os.path.join(base_path, "fake_train")
test_real_path = os.path.join(base_path, "real_test")
test_fake_path = os.path.join(base_path, "fake_test")

# Function to load images and create labels
def load_images(folder_path, label, img_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            try:
                img = imread(img_path)
                if len(img.shape) == 2:  # Convert grayscale to RGB
                    img = np.stack((img,)*3, axis=-1)
                img = resize(img, img_size)  # Resize to 224x224
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

# Load training images
print("Loading real training images...")
real_train_images, real_train_labels = load_images(train_real_path, 1)  # 1 for real
print("Loading fake training images...")
fake_train_images, fake_train_labels = load_images(train_fake_path, 0)  # 0 for fake

# Combine real and fake training images
X_train = np.concatenate((real_train_images, fake_train_images))
y_train = np.concatenate((real_train_labels, fake_train_labels))

# Load test images
print("Loading real test images...")
real_test_images, real_test_labels = load_images(test_real_path, 1)
print("Loading fake test images...")
fake_test_images, fake_test_labels = load_images(test_fake_path, 0)

# Combine real and fake test images
X_test = np.concatenate((real_test_images, fake_test_images))
y_test = np.concatenate((real_test_labels, fake_test_labels))

# Shuffle training and test sets
train_indices = np.arange(len(X_train))
np.random.shuffle(train_indices)
X_train = X_train[train_indices]
y_train = y_train[train_indices]

test_indices = np.arange(len(X_test))
np.random.shuffle(test_indices)
X_test = X_test[test_indices]
y_test = y_test[test_indices]

# Print dataset statistics
print(f"\nTraining set shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"\nClass distribution in training set - Real: {sum(y_train)}, Fake: {len(y_train)-sum(y_train)}")
print(f"Class distribution in test set - Real: {sum(y_test)}, Fake: {len(y_test)-sum(y_test)}")

**Data Augmentation**

In [ ]:
# Create data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No augmentation for validation/test data
test_datagen = ImageDataGenerator()

# Create data generators
batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

**Model Implementation**

In [ ]:
class Meso4:
    def __init__(self, learning_rate=0.001, img_width=224, img_height=224):
        self.img_width = img_width
        self.img_height = img_height
        self.model = self.init_model()
        optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    def init_model(self):
        x = Input(shape=(self.img_width, self.img_height, 3))

        # First convolutional block
        x1 = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

        # Second convolutional block
        x2 = Conv2D(8, (5, 5), padding='same', activation='relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

        # Third convolutional block
        x3 = Conv2D(16, (5, 5), padding='same', activation='relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

        # Fourth convolutional block
        x4 = Conv2D(16, (5, 5), padding='same', activation='relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

        # Fully connected layers
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation='sigmoid')(y)

        return Model(inputs=x, outputs=y)

    def fit(self, train_generator, epochs=50, validation_data=None):
        history = self.model.fit(
            train_generator,
            epochs=epochs,
            validation_data=validation_data,
            verbose=1
        )
        return history

    def evaluate(self, test_generator):
        return self.model.evaluate(test_generator, verbose=1)

    def predict(self, X):
        return self.model.predict(X)

# Initialize the model
meso_model = Meso4(learning_rate=0.001, img_width=224, img_height=224)
meso_model.model.summary()

**Model Training**

In [ ]:
# Define callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

# Train the model
epochs = 50
history = meso_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

**Visualization**

In [ ]:

# Evaluate the model
test_loss, test_acc = meso_model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Make predictions
y_pred_prob = meso_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate all metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)  # Default average='binary' for binary classification
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\n=== Comprehensive Evaluation Metrics ===")
print(f"Accuracy:  {accuracy:.4f}")  # Overall correctness
print(f"Precision: {precision:.4f}") # True positives / (True positives + False positives)
print(f"Recall:    {recall:.4f}")    # True positives / (True positives + False negatives)
print(f"F1 Score:  {f1:.4f}")        # Harmonic mean of precision and recall

# Confusion Matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Fake (0)', 'Real (1)'],
                yticklabels=['Fake (0)', 'Real (1)'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

plot_confusion_matrix(y_test, y_pred)

# ROC Curve
def plot_roc_curve(y_true, y_prob):
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

plot_roc_curve(y_test, y_pred_prob)